# **Statement**

A flat plat 70 cm long and 1.0 m wide is placed in a wind tunnel where the flow conditions are ` M = 3, p = 1/20 atm` , and `T = - 40ºC`. How much cooling must be used to maintain the plate temperature at 35 ºC?

## Set known variables

In [8]:
mach_number = 3 
pressure = (1 / 20) * 101325 # Pa
T = -40 + 273.15 # K

In [2]:
def calcule_acoustic_velocity(specific_weight, R_gas, free_stream_temperature, g_c = 1):
    return (specific_weight * R_gas * free_stream_temperature * g_c) ** (1/2)

In [9]:
from pyfluids import Fluid, FluidsList
from pprint import pprint

water_vapour = Fluid(FluidsList.Water).dew_point_at_pressure(pressure)
pprint(water_vapour.as_dict())  # 2079.937085633241

{'compressibility': 0.997791599005916,
 'conductivity': 0.019007777092064106,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 0.0359227511750955,
 'dynamic_viscosity': 9.960256427118757e-06,
 'enthalpy': 2561147.545630408,
 'entropy': 8389.11286159657,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 2420115.7528452184,
 'kinematic_viscosity': 0.00027726875312445435,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': TwoPhase,
 'prandtl': 1.0071539662474105,
 'pressure': 5066.25,
 'quality': 100.0,
 'sound_speed': 432.12670358990573,
 'specific_heat': 1922.0145814415368,
 'specific_volume': 27.83751152927505,
 'surface_tension': 0.07078736933054516,
 'temperature': 33.10874085726658,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsius

In [1]:
import cantera as ct
import math
import numpy as np


def equilSoundSpeeds(gas, rtol=1.0e-6, max_iter=5000):
    """
    Returns a tuple containing the equilibrium and frozen sound speeds for a
    gas with an equilibrium composition.  The gas is first set to an
    equilibrium state at the temperature and pressure of the gas, since
    otherwise the equilibrium sound speed is not defined.
    """

    # set the gas to equilibrium at its current T and P
    gas.equilibrate('TP', rtol=rtol, max_iter=max_iter)

    # save properties
    s0 = gas.s
    p0 = gas.P
    r0 = gas.density

    # perturb the pressure
    p1 = p0*1.0001

    # set the gas to a state with the same entropy and composition but
    # the perturbed pressure
    gas.SP = s0, p1

    # frozen sound speed
    afrozen = math.sqrt((p1 - p0)/(gas.density - r0))

    # now equilibrate the gas holding S and P constant
    gas.equilibrate('SP', rtol=rtol, max_iter=max_iter)

    # equilibrium sound speed
    aequil = math.sqrt((p1 - p0)/(gas.density - r0))

    # check against the built-in sound speed function
    afrozen2 = gas.sound_speed

    return aequil, afrozen, afrozen2


# test program
if __name__ == "__main__":
    gas = ct.Solution('gri30.yaml')
    gas.X = 'CH4:1.00, O2:2.0, N2:7.52'
    T_range = np.arange(300, 2901, 100)
    for T in T_range:
        gas.TP = T, ct.one_atm
        print(T, equilSoundSpeeds(gas))

300 (351.81897215910766, 351.8225040320234, 351.8256504699811)
400 (404.3283414405797, 404.60713567095115, 404.61068703239766)
500 (450.60495699772054, 450.5397010505711, 450.54335869692443)
600 (493.222522207315, 491.4792382523635, 491.4832718638334)
700 (531.9640558740574, 528.5737908364041, 528.5780360656653)
800 (565.8012509731329, 562.6519893359572, 562.6563486884804)
900 (594.791258636716, 594.3886927199748, 594.3928366122566)
1000 (623.1824547571716, 623.1967317459704, 624.3875595050662)
1100 (652.9331035261605, 653.0237884115232, 653.0280736088952)
1200 (680.2903461483402, 680.3796036866751, 680.3839827317228)
1300 (706.4795733989242, 706.6231858627382, 706.6276554854818)
1400 (731.5764695331436, 731.8946276516258, 731.8991799509749)
1500 (755.5870779532942, 756.3126665962858, 756.3172856209592)
1600 (778.4541581106566, 779.9838959868171, 779.988553975083)
1700 (800.0747674178257, 803.0114202739029, 803.0160744864796)
1800 (820.3499970929598, 825.5030146371989, 825.507606985941